# Visualização da Informação
## Escola de Matemática Aplicada - Fundação Getúlio Vargas
## Mestrado em Modelagem Matemática

Aluno: Gianlucca Devigili<br>
Github do projeto: https://github.com/GDevigili/information-visualization-homeworks

## Trabalho 2: Análise e reprodução de uma visualização reconhecida ou relevante historicamente

+ Parte1: Encontrar os dados (compartilhar referências de dados no slack)
+ Parte2: Fazer uma análise de qual seria a função pretendida com a visualização proposta. 
+ Parte3: Fazer uma reprodução da visualização escolhida utilizando uma ferramenta computacional atual (de preferência a mesma escolhida por vocês no trabalho 1)
+ Parte4: Propor alguma modificação (fundamentando conceitualmente) na visualização proposta. Exemplo: Incluir anotação, incluir interatividade, modificar título ou legenda, adicionar informação, etc.

In [19]:
import pandas as pd
import altair as alt
import plotly.express as px
from plotly.subplots import make_subplots


import vega_datasets

A visualização escolhida foi a de **Florence Nightingale** referente às __causas de morte na guerra da Crimeia__ (1853-1856)


![](https://wp-assets.highcharts.com/www-highcharts-com/blog/wp-content/uploads/2015/05/30154347/Nightingale%E2%80%99s-diagram-1.jpg)

## (1) Aquisição dos dados

Para importar os dados, utilizei a biblioteca `vega_datasets`. <br>Inicialmente eu havia realizado um web-scrapping dos dados ([como pode ser visto neste commit]([https://github.com/GDevigili/information-visualization-homeworks/commit/a2c97beb6c5c5f1e174c854f094720b09437893a#diff-f485367de3abdc640726199ca3360f1fdfc1043bf3abdde1ecf95faa3556adf2)), porém ao plotar o gráfico percebi que os dados da fonte que eu peguei estavam errados e não reproduziam o gráfico de Nightingale, então preferi usar a biblioteca.

In [14]:
# Carrega os dados
df_crimea = vega_datasets.data.crimea()
df_crimea

,date,wounds,other,disease
0,1854-04-01,0,110,110
1,1854-05-01,0,95,105
2,1854-06-01,0,40,95
3,1854-07-01,0,140,520
4,1854-08-01,20,150,800
5,1854-09-01,220,230,740
6,1854-10-01,305,310,600
7,1854-11-01,480,290,820
8,1854-12-01,295,310,1100
9,1855-01-01,230,460,1440


## (2) Análise da Função da Visualização


## (3) Reprodução da Visualização

Para reproduzir o gráfico, precisamos dividi os dados em dois intervalos de tempo, sendo o primeiro indo de abril de 1854 até março de 1855 e o segundo de abril de 1855 até março de 1856.<br>
Após isso, usei o método `pd.melt` para transformar o dataset de modo que ele tenha 3 colunas: `Date`, `Death` e `Cause`. Os dados então ficam da maneira apresentada abaixo:

In [15]:
# Formata a data no formato 'Apr 1854'
df_crimea['date'] = [date.strftime('%b %Y') for date in df_crimea['date']]

df1 = pd.melt(
        df_crimea[:12], 
        id_vars = ['date'], 
        value_vars = causes,
        var_name = 'Cause', value_name = 'Deaths'
        )
df2 = pd.melt(
        df_crimea[12:], 
        id_vars = ['date'], 
        value_vars = causes,
        var_name = 'Cause', value_name = 'Deaths'
        )
df1.head()

,date,Cause,Deaths
0,Apr 1854,other,110
1,May 1854,other,95
2,Jun 1854,other,40
3,Jul 1854,other,140
4,Aug 1854,other,150


In [26]:
colors = ['#fac0bd', '#4c3933', '#b0c3d1']

fig1 = px.bar_polar(
    df1, 
    r = 'Deaths',
    color = 'Cause', 
    theta = 'date',
    start_angle = 165,
    color_discrete_sequence = colors,
    template = 'simple_white'
)

fig1.show()

fig2 = px.bar_polar(
    df2, 
    r = 'Deaths',
    color = 'Cause', 
    theta = 'date',
    start_angle = 165,
    color_discrete_sequence = colors,
    template = 'simple_white'
)

fig2.show()

## (4) Modificações Propostas